In [1]:
import os
from openai import OpenAI as OpenAIClient
from dotenv import load_dotenv
from llama_parse import LlamaParse
from llama_index.core.node_parser import HierarchicalNodeParser
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI as LlamaOpenAI

import os
from llama_index.core.node_parser import HierarchicalNodeParser
from llama_index.core.node_parser import get_leaf_nodes
from llama_parse import LlamaParse

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY is not set in .env file")

llm = LlamaOpenAI(model="gpt-4o", api_key=openai_api_key)
Settings.llm = llm
openai_client = OpenAIClient(api_key=openai_api_key)

llama_cloud_api_key = os.getenv("LLAMA_CLOUD_API_KEY")
if not llama_cloud_api_key:
    raise ValueError("LLAMA_CLOUD_API_KEY is not set in .env file")

ValueError: OPENAI_API_KEY is not set in .env file

In [5]:
llama_parser = LlamaParse(page_prefix="START OF PAGE: {pageNumber}\n",page_suffix="\nEND OF PAGE: {pageNumber}",api_key="",verbose=True,result_type="markdown")
# Load the document again with the new parser settings
file_name = "sample_runbook.docx"
extra_info = {"file_name": file_name, "file_type": "docx"}

documents = llama_parser.load_data("./runbooks/sample_runbook.docx", extra_info=extra_info)
for i, doc in enumerate(documents, start=1):
    doc.metadata["page_number"] = i



Started parsing the file under job_id f5ba448b-50cd-486c-ad1f-ee21a0ce1c63


In [ ]:
print("--- Full Markdown Content of Document ---")
print(documents[0].text[:500].strip() + "...\n")
print("---------------------------------------")

In [7]:
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048, 512, 128]
)

In [9]:
node_parser = MarkdownElementNodeParser(llm=llm, num_workers=15)

In [10]:
nodes = node_parser.get_nodes_from_documents(documents)
leaf_nodes = get_leaf_nodes(nodes)
print(f"Total number of nodes parsed: {len(nodes)}\n")


all_nodes_dict = {node.node_id: node for node in nodes}


relevant_nodes = []
keywords = ["DR", "disaster", "recovery", "failover", "fallback", "redundant"]

for node in leaf_nodes:
    if any(kw.lower() in node.text.lower() for kw in keywords):
        relevant_nodes.append(node)

print(f"\nFound {len(relevant_nodes)} relevant nodes based on keywords.\n" + "="*50)


for i, node in enumerate(relevant_nodes, start=1):
    if i<=10:
        print(f"--- Relevant Node {i} ---")
        # print(f"Text Snippet: {node.text[:100].strip()}...")

        doc_name = node.metadata.get('file_name', 'N/A')
        print(f"Document Name: {doc_name}")

        section_name = "N/A (Top Level)"
        if node.parent_node:
            parent_id = node.parent_node.node_id
            
            parent_node_obj = all_nodes_dict.get(parent_id)
            
            if parent_node_obj:
                section_name = parent_node_obj.text.strip()
                
        print(f"Section: {section_name}")


        page_num = node.metadata.get('page_number', 'N/A (Not available for .docx)')
        print(f"Page Number: {page_num}")
        
        print(f"Original Metadata Dict: {node.metadata}")
        print("-" * 25 + "\n")

2it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 986.20it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
3it [00:00, ?it/s]
4it [00:00, ?it/s]
0it [00:00, ?it/s]

Total number of nodes parsed: 46


Found 15 relevant nodes based on keywords.
--- Relevant Node 1 ---
Document Name: sample_runbook.docx
Section: N/A (Top Level)
Page Number: 2
Original Metadata Dict: {'file_name': 'sample_runbook.docx', 'file_type': 'docx', 'page_number': 2}
-------------------------

--- Relevant Node 2 ---
Document Name: sample_runbook.docx
Section: N/A (Top Level)
Page Number: 3
Original Metadata Dict: {'file_name': 'sample_runbook.docx', 'file_type': 'docx', 'page_number': 3, 'col_schema': 'Column: Application\nType: string\nSummary: Names of different applications or systems.\n\nColumn: Production IP\nType: string\nSummary: IP addresses used in the production environment.\n\nColumn: DR IP\nType: string\nSummary: IP addresses used in the disaster recovery environment.'}
-------------------------

--- Relevant Node 3 ---
Document Name: sample_runbook.docx
Section: N/A (Top Level)
Page Number: 3
Original Metadata Dict: {'file_name': 'sample_runbook.docx', 'file_type